In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pickle as pkl
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
from xarray import DataArray
from PIL import Image
import xml.etree.ElementTree as ET
from schema_agents.provider.openai_api import retry
import shutil

import asyncio
from schema_agents.role import Role
from schema_agents.schema import Message
from pydantic import BaseModel, Field, validator

from bioimageio_chatbot.image_processor import *


2023-12-10 19:57:23.193 | INFO     | schema_agents.config:__init__:44 - Config loading done.
2023-12-10 19:57:23.193 | INFO     | schema_agents.config:__init__:55 - Set OPENAI_API_BASE in case of network issues
/Users/gkreder/opt/miniconda3/envs/novel-designer/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_path = "../tmp/image_db/"
image_processor = ImageProcessor()
db = get_torch_db(db_path, image_processor)

In [3]:
cellpose_dir = os.path.join(db_path, 'cellpose_train-set_representatives')
cellpose_temp_dir = os.path.join(db_path, "cellpose_tmp")
if not os.path.exists(cellpose_dir):
    os.makedirs(cellpose_temp_dir, exist_ok=True)
    cellpose_link = "https://drive.google.com/drive/folders/1js8urhijmL4h60JxTUDAE7AWmExEoKoP?usp=drive_link"
    cellpose_drive_id = "1kEy9DHxkGeQHQDQco8CyaL1KEEPS6NYY"
    cellpose_zip_file = os.path.join(db_path, "cellpose_train_reps.zip")
    wget_cmd = f"wget --no-check-certificate 'https://docs.google.com/uc?export=download&id={cellpose_drive_id}' -O {cellpose_zip_file}"
    os.system(wget_cmd)
    unzip_cmd = f"unzip {cellpose_zip_file} -d {cellpose_temp_dir}"
    os.system(unzip_cmd)
    shutil.move(os.path.join(cellpose_temp_dir, "cellpose_train-set_representatives"), db_path)
    shutil.rmtree(cellpose_temp_dir, ignore_errors=True)
# axis_guesser = Role(name = "AxisGuesser",
#                 profile = "Axis Guesser",
#                 goal="Your goal as AxisGuesser is read the shapes of input images and guess their axis labels using common sense.",
#             constraints=None,
#             actions=[guess_image_axes, guess_all_axes])
# event_bus = axis_guesser.get_event_bus()
# event_bus.register_default_events()
# input_images = [os.path.join(cellpose_dir, x) for x in os.listdir(cellpose_dir) if x.endswith('.png')]
# image_processor = ImageProcessor()
# guessed_axes = []
# shapes = []
for input_image_path in input_images:
    input_image = image_processor.read_image(input_image_path)
    shapes.append(input_image.shape)
message_input = UnlabeledImages(unlabeled_images = [UnlabeledImage(shape = s) for s in shapes])
m = Message(content = 'guess the image axes for each image in the list', data = message_input, role = 'User')
responses = await axis_guesser.handle(m)
guessed_axes = [''.join(x.axes.labels) for x in responses[0].data.labeled_images]
svg_content = create_svg_table_unknown(input_images, shapes, guessed_axes)
with open(os.path.join(db_path, 'cellpose_guessed_axes.svg'), "w") as file:
    file.write(svg_content)

In [9]:
input_image.shape

(385, 512, 3)